pip install ftfy regex tqdm

pip install git+https://github.com/openai/CLIP.git

In [1]:
import torch
import clip
from PIL import Image

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
clip.available_models()

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'ViT-B/32', 'ViT-B/16']

In [4]:
model, preprocess = clip.load("ViT-B/16", device=device, jit=False)

100%|███████████████████████████████████████| 335M/335M [00:49<00:00, 7.09MiB/s]


In [14]:
image = preprocess(Image.open("images/dog-bird-2.jpg")).unsqueeze(0).to(device)
image.shape

torch.Size([1, 3, 224, 224])

In [15]:
labels = ["a dog", "a cat", "a bird", "a plane"]
text = clip.tokenize(labels).to(device)
text.shape

torch.Size([4, 77])

In [16]:
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()
lw = len(max(labels, key=lambda x: len(x)))
row_fmt = '{:>{lw}s}: {:>6.2f}'
print(f'{"Label":>{lw}s}: {"Prob":>6}')
for l, p in zip(labels, probs[0]):
    print(f'{l:>{lw}s}: {100*p:>6.2f}')

  Label:   Prob
  a dog:  72.17
  a cat:   1.32
 a bird:  18.53
a plane:   7.97
